# 파이썬 클래스 구조 훈련 노트북

이 노트북은 추상 클래스, 상속, 메서드 오버라이딩, LLM Wrapper 구조 연습을 위한 단계별 실습으로 구성되어 있습니다.

## 1. 추상 클래스와 상속 기본

- 추상 클래스(ABC)와 추상 메서드(@abstractmethod) 사용법을 익혀봅니다.

In [2]:
from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def speak(self):
        pass

class Dog(Animal):
    def speak(self):
        print("멍멍!")

# 실습: Dog 클래스를 인스턴스화하고 speak()를 호출해보세요.
dog = Dog()
dog.speak()

멍멍!


## 2. LLM_Wrapper 추상 클래스 설계

- LLM_Wrapper 추상 클래스를 만들어보고, 필수 메서드와 속성을 정의해봅니다.

In [3]:
from abc import ABC, abstractmethod
from collections.abc import MutableSequence
from typing import TypedDict

class LLM_Wrapper(ABC):
    MessageEntry = TypedDict('MessageEntry', {'role': str, 'content': str})
    system_prompt: str
    user_prompt: str
    temperature: float = 0.5
    __msg: MutableSequence[MessageEntry]

    def __init__(self, system_prompt, user_prompt, temperature=0.5):
        self.system_prompt = system_prompt
        self.user_prompt = user_prompt
        self.temperature = temperature
        self.__msg = []

    def messageSet(self, message: MutableSequence[MessageEntry]):
        self.__msg = message

    def messageAppend(self, role: str, content: str):
        self.__msg.append({"role": role, "content": content})

    def messageGet(self) -> MutableSequence[MessageEntry]:
        return self.__msg

    @abstractmethod
    def getResult(self):
        pass

## 3. LLM_Wrapper 상속 클래스 연습

- LLM_Wrapper를 상속받아 실제 LLM API를 흉내내는 클래스를 만들어봅니다.

In [4]:
class FakeLLM_Wrapper(LLM_Wrapper):
    def getResult(self):
        # 실제 LLM 호출 대신 메시지 내용을 합쳐서 반환
        messages = self.messageGet()
        return '\n'.join([f"{m['role']}: {m['content']}" for m in messages])

# 실습: FakeLLM_Wrapper 인스턴스를 만들고 메시지를 추가한 뒤 getResult를 호출해보세요.
wrapper = FakeLLM_Wrapper("시스템 프롬프트", "유저 프롬프트")
wrapper.messageAppend("assistant", "안녕하세요!")
print(wrapper.getResult())

assistant: 안녕하세요!


## 4. 실제 LLM Wrapper 구조 연습

- 실제로 OpenAI, Ollama, Claude, Gemini 등 다양한 래퍼 클래스를 만들어보고 싶다면, 위 구조를 참고해서 직접 만들어보세요.
- 각 래퍼 클래스는 getResult 메서드에서 실제 API 호출 코드를 작성하면 됩니다.

아래는 연습용 예시입니다.

In [5]:
# 예시: OpenAI 래퍼 클래스 구조 (실제 API 호출은 생략)
class OpenAI_Wrapper(LLM_Wrapper):
    def getResult(self):
        # 실제로는 OpenAI API를 호출해야 함
        return f"[OpenAI 응답] {self.user_prompt}"

# 실습: OpenAI_Wrapper 인스턴스를 만들고 getResult를 호출해보세요.
gpt = OpenAI_Wrapper("너는 친절한 챗봇이야.", "오늘 날씨 어때?")
print(gpt.getResult())

[OpenAI 응답] 오늘 날씨 어때?
